# CogniSense Functional Testing

This notebook runs comprehensive functional tests for the CogniSense project.

**Run this in Google Colab to verify all features work correctly.**

---

## Setup

In [ ]:
# Clone repository if running in Colab
import os
import sys

if 'google.colab' in sys.modules:
    if not os.path.exists('AI4Alzheimers'):
        !git clone https://github.com/Arnavsharma2/AI4Alzheimers.git
    %cd AI4Alzheimers
    !git checkout claude/review-drive-folder-01KHZ15iXzj7ZQnkH8rNKb62
    print("✓ Repository cloned and checked out")
else:
    print("✓ Running locally")

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
print("✓ Dependencies installed")

## Test 1: Import All Modules

In [ ]:
print("Testing imports...\n")

# Core dependencies
import torch
print(f"✓ PyTorch {torch.__version__}")

import transformers
print(f"✓ Transformers {transformers.__version__}")

import numpy as np
print(f"✓ NumPy {np.__version__}")

import sklearn
print(f"✓ scikit-learn {sklearn.__version__}")

# CogniSense modules
from src.models.eye_model import EyeTrackingModel
from src.models.typing_model import TypingModel
from src.models.drawing_model import ClockDrawingModel
from src.models.gait_model import GaitModel
from src.fusion.fusion_model import MultimodalFusionModel
print("\n✓ All CogniSense models imported")

from src.data_processing.synthetic_data_generator import (
    EyeTrackingGenerator,
    TypingDynamicsGenerator,
    ClockDrawingGenerator,
    GaitDataGenerator,
    generate_synthetic_dataset
)
print("✓ All data generators imported")

from src.utils.training_utils import compute_metrics, EarlyStopping, train_epoch, evaluate
from src.utils.visualization import plot_roc_curve, plot_confusion_matrix
print("✓ All utilities imported")

print("\n✅ ALL IMPORTS SUCCESSFUL")

## Test 2: Synthetic Data Generation

In [ ]:
print("Generating synthetic data...\n")

# Eye tracking data
eye_gen = EyeTrackingGenerator()
eye_control = eye_gen.generate_sequence(is_alzheimers=False)
eye_ad = eye_gen.generate_sequence(is_alzheimers=True)
print(f"✓ Eye tracking data: Control {eye_control.shape}, AD {eye_ad.shape}")

# Typing data
typing_gen = TypingDynamicsGenerator()
typing_control = typing_gen.generate_sequence(is_alzheimers=False)
typing_ad = typing_gen.generate_sequence(is_alzheimers=True)
print(f"✓ Typing data: Control {typing_control.shape}, AD {typing_ad.shape}")

# Clock drawing
drawing_gen = ClockDrawingGenerator()
drawing_control = drawing_gen.generate_image(is_alzheimers=False)
drawing_ad = drawing_gen.generate_image(is_alzheimers=True)
print(f"✓ Clock drawing: Control {drawing_control.shape}, AD {drawing_ad.shape}")

# Gait data
gait_gen = GaitDataGenerator()
gait_control = gait_gen.generate_sequence(is_alzheimers=False)
gait_ad = gait_gen.generate_sequence(is_alzheimers=True)
print(f"✓ Gait data: Control {gait_control.shape}, AD {gait_ad.shape}")

print("\n✅ SYNTHETIC DATA GENERATION SUCCESSFUL")

## Test 3: Model Instantiation

In [ ]:
print("Creating models...\n")

# Individual models
eye_model = EyeTrackingModel()
print(f"✓ Eye model: {sum(p.numel() for p in eye_model.parameters())} parameters")

typing_model = TypingModel()
print(f"✓ Typing model: {sum(p.numel() for p in typing_model.parameters())} parameters")

print("  Loading ClockDrawingModel (downloading pretrained weights...)")
drawing_model = ClockDrawingModel()
print(f"✓ Drawing model: {sum(p.numel() for p in drawing_model.parameters())} parameters")

gait_model = GaitModel()
print(f"✓ Gait model: {sum(p.numel() for p in gait_model.parameters())} parameters")

# Fusion model
fusion_model = MultimodalFusionModel(fusion_type='attention')
print(f"✓ Fusion model: {sum(p.numel() for p in fusion_model.parameters())} parameters")

print("\n✅ ALL MODELS INSTANTIATED SUCCESSFULLY")

## Test 4: Forward Passes

In [ ]:
print("Testing forward passes...\n")

# Set models to eval mode
eye_model.eval()
typing_model.eval()
drawing_model.eval()
gait_model.eval()

with torch.no_grad():
    # Eye model
    eye_input = torch.FloatTensor(eye_ad).unsqueeze(0)
    eye_output = eye_model(eye_input)
    print(f"✓ Eye model: {eye_input.shape} → {eye_output.shape}")
    assert eye_output.shape == (1, 64), f"Expected (1, 64), got {eye_output.shape}"
    
    # Typing model
    typing_input = torch.FloatTensor(typing_ad).unsqueeze(0)
    typing_output = typing_model(typing_input)
    print(f"✓ Typing model: {typing_input.shape} → {typing_output.shape}")
    assert typing_output.shape == (1, 64), f"Expected (1, 64), got {typing_output.shape}"
    
    # Drawing model
    drawing_input = torch.FloatTensor(drawing_ad).permute(2, 0, 1).unsqueeze(0) / 255.0
    drawing_output = drawing_model(drawing_input)
    print(f"✓ Drawing model: {drawing_input.shape} → {drawing_output.shape}")
    assert drawing_output.shape == (1, 64), f"Expected (1, 64), got {drawing_output.shape}"
    
    # Gait model
    gait_input = torch.FloatTensor(gait_ad).unsqueeze(0).permute(0, 2, 1)
    gait_output = gait_model(gait_input)
    print(f"✓ Gait model: {gait_input.shape} → {gait_output.shape}")
    assert gait_output.shape == (1, 64), f"Expected (1, 64), got {gait_output.shape}"

print("\n✅ ALL FORWARD PASSES SUCCESSFUL")

## Test 5: Multimodal Fusion

In [ ]:
print("Testing multimodal fusion...\n")

fusion_model.eval()

with torch.no_grad():
    # Prepare inputs
    eye_in = torch.FloatTensor(eye_ad).unsqueeze(0)
    typing_in = torch.FloatTensor(typing_ad).unsqueeze(0)
    drawing_in = torch.FloatTensor(drawing_ad).permute(2, 0, 1).unsqueeze(0) / 255.0
    gait_in = torch.FloatTensor(gait_ad).unsqueeze(0).permute(0, 2, 1)
    
    # Forward pass with attention
    output, attention = fusion_model(
        speech_audio=None,
        speech_text=None,
        eye_tracking=eye_in,
        typing_dynamics=typing_in,
        clock_drawing=drawing_in,
        gait_data=gait_in,
        return_attention=True
    )
    
    print(f"✓ Fusion output shape: {output.shape}")
    print(f"✓ Attention weights shape: {attention.shape}")
    
    # Verify attention sums to 1
    attention_sum = attention.sum().item()
    print(f"✓ Attention sum: {attention_sum:.6f} (should be ~1.0)")
    assert abs(attention_sum - 1.0) < 1e-5, f"Attention doesn't sum to 1: {attention_sum}"
    
    # Calculate risk score
    risk_score = torch.sigmoid(output).item()
    print(f"✓ AD risk score: {risk_score:.4f} (0-1, higher = more likely AD)")
    
    # Display attention weights
    print("\nAttention weights:")
    modalities = ['Speech', 'Eye', 'Typing', 'Drawing', 'Gait']
    for i, (mod, weight) in enumerate(zip(modalities, attention[0])):
        bar = '█' * int(weight.item() * 50)
        print(f"  {mod:10s}: {weight.item():.4f} {bar}")

print("\n✅ MULTIMODAL FUSION SUCCESSFUL")

## Test 6: Dataset Creation

In [ ]:
print("Creating dataset...\n")

from src.data_processing.dataset import MultimodalAlzheimerDataset, custom_collate_fn
from torch.utils.data import DataLoader

# Generate small dataset
data_dict = generate_synthetic_dataset(
    num_samples=20,
    modalities=['eye', 'typing', 'drawing', 'gait']
)
print(f"✓ Generated {len(data_dict['labels'])} samples")

# Create dataset
dataset = MultimodalAlzheimerDataset(
    data_dict,
    modalities=['eye', 'typing', 'drawing', 'gait']
)
print(f"✓ Dataset size: {len(dataset)}")

# Test single sample
sample = dataset[0]
print(f"✓ Sample keys: {list(sample.keys())}")
print(f"  Label: {sample['label']}")

# Create DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=custom_collate_fn
)
print(f"✓ DataLoader created with batch_size=4")

# Test batch
batch = next(iter(dataloader))
print(f"✓ Batch loaded:")
print(f"  Eye: {len(batch['eye'])} samples")
print(f"  Typing: {len(batch['typing'])} samples")
print(f"  Drawing: {batch['drawing'].shape}")
print(f"  Gait: {len(batch['gait'])} samples")
print(f"  Labels: {batch['label'].shape}")

print("\n✅ DATASET CREATION SUCCESSFUL")

## Test 7: Training Utilities

In [ ]:
print("Testing training utilities...\n")

# Test metrics computation
y_true = np.array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1])
y_prob = np.array([0.1, 0.2, 0.8, 0.9, 0.3, 0.7, 0.4, 0.2, 0.85, 0.6])

metrics = compute_metrics(y_true, y_pred, y_prob)
print("✓ Metrics computed:")
for key, value in metrics.items():
    print(f"  {key:15s}: {value:.4f}")

# Test early stopping
early_stop = EarlyStopping(patience=3, mode='max')
scores = [0.70, 0.75, 0.80, 0.85, 0.84, 0.83, 0.82, 0.81]
stopped_at = None

for epoch, score in enumerate(scores):
    if early_stop(score):
        stopped_at = epoch
        break

print(f"\n✓ Early stopping triggered at epoch {stopped_at}")
print(f"  Best score: {early_stop.best_score:.4f}")

print("\n✅ TRAINING UTILITIES SUCCESSFUL")

## Test 8: Quick Training Test

In [ ]:
print("Running quick training test (5 epochs)...\n")

# Create small training dataset
train_data = generate_synthetic_dataset(num_samples=50, modalities=['eye', 'typing', 'gait'])
train_dataset = MultimodalAlzheimerDataset(train_data, modalities=['eye', 'typing', 'gait'])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=custom_collate_fn)

# Create simple model
test_model = EyeTrackingModel()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(test_model.parameters(), lr=0.001, weight_decay=0.01)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

test_model.to(device)

# Training loop
for epoch in range(5):
    test_model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        # Get eye tracking data only
        inputs = [torch.FloatTensor(x).to(device) for x in batch['eye']]
        labels = batch['label'].float().to(device)
        
        optimizer.zero_grad()
        
        # Forward passes for each sample (variable length)
        outputs = []
        for inp in inputs:
            out = test_model(inp.unsqueeze(0))
            outputs.append(out)
        
        # For simplicity, just use first output dimension
        outputs = torch.cat([o[:, 0] for o in outputs])
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += len(labels)
    
    avg_loss = epoch_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch {epoch+1}/5: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")

print("\n✅ QUICK TRAINING TEST SUCCESSFUL")

## Test 9: Visualization

In [ ]:
print("Testing visualization functions...\n")

import matplotlib.pyplot as plt
from src.utils.visualization import plot_confusion_matrix, plot_attention_heatmap

# Generate fake predictions
np.random.seed(42)
y_true = np.random.randint(0, 2, 100)
y_pred = np.random.randint(0, 2, 100)

# Confusion matrix
fig = plot_confusion_matrix(y_true, y_pred, normalize=True)
plt.show()
print("✓ Confusion matrix plotted")

# Attention heatmap
attention_weights = np.random.rand(10, 5)
attention_weights = attention_weights / attention_weights.sum(axis=1, keepdims=True)
modality_names = ['Speech', 'Eye', 'Typing', 'Drawing', 'Gait']

fig = plot_attention_heatmap(attention_weights, modality_names)
plt.show()
print("✓ Attention heatmap plotted")

print("\n✅ VISUALIZATION SUCCESSFUL")

## Summary

In [ ]:
print("\n" + "="*60)
print("  🎉 ALL FUNCTIONAL TESTS PASSED! 🎉")
print("="*60)

print("\nTest Results:")
print("  ✅ Module imports")
print("  ✅ Synthetic data generation")
print("  ✅ Model instantiation")
print("  ✅ Forward passes")
print("  ✅ Multimodal fusion")
print("  ✅ Dataset creation")
print("  ✅ Training utilities")
print("  ✅ Quick training")
print("  ✅ Visualization")

print("\n" + "="*60)
print("  CogniSense is fully functional! 🚀")
print("="*60)